# Image Generation and Background Replacement using Stable Diffusion

In [1]:
import torch
from diffusers import StableDiffusionPipeline
import numpy as np
from PIL import Image

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]